#### Importing Libraries

In [2]:
import datetime
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass

#### Making the Holiday class

In [76]:
class Holiday:

    def __init__(self, name, date):
            
        date_format = '%Y-%m-%d'
        date = datetime.datetime.strptime(date, date_format)
        self._name = name 
        self._date = date

    def __str__(self):
        return f'The holiday is {self._name} and the date is {self._date}'
    

Christmas = Holiday('Christmas', '1999-01-01')
Jeffday = Holiday('Jeff', '1999-11-18')
print(Christmas)
print(vars(Christmas))
print(Jeffday)
print(vars(Jeffday))

The holiday is Christmas and the date is 1999-01-01 00:00:00
{'_name': 'Christmas', '_date': datetime.datetime(1999, 1, 1, 0, 0)}
The holiday is Jeff and the date is 1999-11-18 00:00:00
{'_name': 'Jeff', '_date': datetime.datetime(1999, 11, 18, 0, 0)}


In [77]:
class HolidayList:
    
    innerHolidays = []
    
    def __init__(self, holidayObj):
        self.holidayObj = holidayObj

    def addHoliday(self, holidayObj):
        
        self.innerHolidays.append(vars(holidayObj))
        return self.innerHolidays

    def __str__(self):
        return f'The Holidays are as follows: {self.innerHolidays}'
    
    def __repr__(self):
        return f'the holiday is {self.holidayObj}, holiday list: {self.innerHolidays}'
    
    def findHoliday(HolidayName, Date):
        return HolidayName, Date

full_list = HolidayList
print(full_list.addHoliday(HolidayList, Christmas))
print(full_list.addHoliday(full_list, Jeffday))

[{'_name': 'Christmas', '_date': datetime.datetime(1999, 1, 1, 0, 0)}]
[{'_name': 'Christmas', '_date': datetime.datetime(1999, 1, 1, 0, 0)}, {'_name': 'Jeff', '_date': datetime.datetime(1999, 11, 18, 0, 0)}]
[{'_name': 'Christmas', '_date': datetime.datetime(1999, 1, 1, 0, 0)}, {'_name': 'Jeff', '_date': datetime.datetime(1999, 11, 18, 0, 0)}, {'_name': 'Thanos', '_date': datetime.datetime(2001, 9, 11, 0, 0)}]
